In [1]:
import sqlite3
import pandas as pd
from pyprojroot import here

**Generate DB's Report**

In [2]:
# Connect to the SQLite database
conn = sqlite3.connect(here('data/travel2.backup.sqlite'))
# Get a list of tables in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)

In [ ]:
# Create a report string
report = "### Database Report\n\n"

# Introduction
report += "This report provides an overview of the database, including its structure, key metrics, and insights derived from the data.\n\n"

# Iterate over each table to extract information
for table_name in tables['name']:
    report += f"#### Table: {table_name}\n\n"
    
    # Get table schema
    schema = pd.read_sql_query(f"PRAGMA table_info({table_name});", conn)
    report += "Columns:\n\n"
    for idx, row in schema.iterrows():
        report += f"- **{row['name']}** ({row['type']})\n"
    
    # Get row count
    row_count = pd.read_sql_query(f"SELECT COUNT(*) as count FROM {table_name};", conn)['count'][0]
    report += f"\nRow Count: {row_count}\n\n"
    
    # Estimate table size (approximate)
    # SQLite doesn't have a direct way to get table size, so we'll estimate it
    total_bytes = pd.read_sql_query(f"PRAGMA page_count;", conn)['page_count'][0] * pd.read_sql_query(f"PRAGMA page_size;", conn)['page_size'][0]
    report += f"Estimated Table Size: {total_bytes / 1024:.2f} KB\n\n"
    
    # Get sample data
    sample_data = pd.read_sql_query(f"SELECT * FROM {table_name} LIMIT 5;", conn)
    report += "Sample Data:\n\n"
    report += sample_data.to_markdown() + "\n\n"
    
    # Summary statistics
    if 'INTEGER' in schema['type'].values or 'REAL' in schema['type'].values:
        summary = pd.read_sql_query(f"SELECT * FROM {table_name};", conn).describe()
        report += "Summary Statistics:\n\n"
        report += summary.to_markdown() + "\n\n"
    
    report += "\n"

# Closing the connection
conn.close()

# Save the report to a markdown file
with open("database_report.md", "w") as file:
    file.write(report)

print("Report generated and saved as 'database_report.md'.")